In [284]:
# Not subclassing prevents using different behaviour in training and in generation. Means dropout is not an option.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
import scipy
from sklearn import preprocessing

%matplotlib inline

2024-01-25 14:24:57.699734: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-25 14:24:57.721986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 14:24:57.722008: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 14:24:57.722023: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-25 14:24:57.726205: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-25 14:24:57.726530: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [286]:
# Get in dataset
attacks = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/csvDataFeaturesTest.csv", sep=";")

In [287]:
attacks = preprocessing.normalize(attacks.to_numpy(), norm="max", axis=0)

In [288]:
attacks.shape

(248278, 22)

In [289]:
# should probably add batch and shuffle

train_dataset = attacks[:int(np.floor(attacks.shape[0]*3/4))]
test_dataset = attacks[int(np.floor(attacks.shape[0]*3/4)):]

# buffer_size = 10000
# batch_size = 50

# train_dataset = (tf.data.Dataset.from_tensor_slices(attacks[:int(np.floor(attacks.shape[0]*3/4))]).shuffle(buffer_size).batch(batch_size))
# test_dataset = (tf.data.Dataset.from_tensor_slices(attacks[int(np.floor(attacks.shape[0]*3/4)):]).shuffle(buffer_size).batch(batch_size))

In [290]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return tf.keras.backend.random_normal(tf.shape(log_var)) * tf.keras.backend.exp(log_var / 2) + mean

In [291]:
codings_size = 100
num_features = attacks[:int(np.floor(attacks.shape[0]*3/4))].shape[1]

In [292]:
# encoder

inputs = tf.keras.layers.Input(shape=[num_features,1])
z = tf.keras.layers.Flatten()(inputs)
z = tf.keras.layers.Dense(150, activation="relu", kernel_initializer="he_normal")(z)
z = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(z)
codings_mean = tf.keras.layers.Dense(codings_size)(z) # mu
codings_log_var = tf.keras.layers.Dense(codings_size)(z) # gamma
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = tf.keras.Model(inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

In [293]:
# decoder

decoder_inputs = tf.keras.layers.Input(shape=[codings_size])
x = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(decoder_inputs)
x = tf.keras.layers.Dense(150, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(num_features*1, activation="sigmoid")(x)
outputs = tf.keras.layers.Reshape([num_features, 1])(x)
variational_decoder = tf.keras.Model(inputs=[decoder_inputs], outputs=[outputs])

In [294]:
# VAE model

_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = tf.keras.Model(inputs=[inputs], outputs=[reconstructions])

In [295]:
# losses

latent_loss = -0.5 * tf.keras.backend.sum(1 + codings_log_var - tf.keras.backend.exp(codings_log_var) - tf.keras.backend.square(codings_mean), axis=-1)
variational_ae.add_loss(tf.keras.backend.mean(latent_loss / float(num_features)))
# variational_ae.add_loss(tf.keras.backend.mean(latent_loss))
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-5)
variational_ae.compile(loss="binary_crossentropy", optimizer=optimizer)

In [296]:
history = variational_ae.fit(train_dataset, train_dataset, epochs=15, batch_size=10)

Epoch 1/15
18621/18621 [==============================] - 15s 776us/step - loss: 0.4292
Epoch 2/15
18621/18621 [==============================] - 15s 783us/step - loss: 0.3167
Epoch 3/15
18621/18621 [==============================] - 14s 777us/step - loss: 0.3057
Epoch 4/15
18621/18621 [==============================] - 15s 779us/step - loss: 0.2991
Epoch 5/15
18621/18621 [==============================] - 14s 778us/step - loss: 0.2946
Epoch 6/15
18621/18621 [==============================] - 15s 783us/step - loss: 0.2899
Epoch 7/15
18621/18621 [==============================] - 15s 785us/step - loss: 0.2870
Epoch 8/15
18621/18621 [==============================] - 15s 781us/step - loss: 0.2851
Epoch 9/15
18621/18621 [==============================] - 15s 784us/step - loss: 0.2837
Epoch 10/15
18621/18621 [==============================] - 15s 782us/step - loss: 0.2823
Epoch 11/15
18621/18621 [==============================] - 15s 780us/step - loss: 0.2812
Epoch 12/15
18621/18621 [=====

In [297]:
predictions = variational_ae.predict(test_dataset)

num_values = test_dataset.shape[0] * test_dataset.shape[1]

scipy.stats.ks_2samp(test_dataset.reshape(num_values,), predictions.reshape(num_values,))

1940/1940 [==============================] - 1s 584us/step


KstestResult(statistic=0.619777523909955, pvalue=0.0, statistic_location=0.0, statistic_sign=1)

In [298]:
scipy.stats.ks_2samp(test_dataset.reshape(num_values,), train_dataset.reshape(train_dataset.shape[0] * train_dataset.shape[1]))

KstestResult(statistic=0.016145012790877433, pvalue=2.633356377928654e-232, statistic_location=0.9596774193548391, statistic_sign=1)